# Basic-Planner

From our own testing, planner works best with more powerful models like `gpt4` but sometimes you might get working plans with cheaper models like `gpt-35-turbo`. We encourage you to implement your own versions of the planner and use different models that fit your user needs.  

Read more about planner [here](https://aka.ms/sk/concepts/planner)

In [1]:
#!python -m pip install semantic-kernel==0.4.5.dev0

In [21]:
import os
endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
deployment = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
api_key = os.environ["AZURE_OPENAI_API_KEY"]
embeddings = os.environ["AZURE_OPENAI_EMBEDDINGS_MODEL_NAME"]
azure_ai_search_api_key = os.environ["AZURE_AISEARCH_API_KEY"]
azure_ai_search_url = os.environ["AZURE_AISEARCH_ENDPOINT"]
#BING_API_KEY = os.environ["BING_API_KEY"]

In [22]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    OpenAIChatCompletion,
    AzureChatCompletion,
)

kernel = sk.Kernel()

   
kernel.add_chat_service(
        "chat_completion",
        AzureChatCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key),
)

## It all begins with an ask

In [23]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas. She speaks French so write it in French.
Convert the text to uppercase"""

### Providing skills to the planner
The planner needs to know what skills are available to it. Here we'll give it access to the `SummarizeSkill` and `WriterSkill` we have defined on disk. This will include many semantic functions, of which the planner will intelligently choose a subset. 

You can also include native functions as well. Here we'll add the TextSkill.

In [24]:
from semantic_kernel.core_skills.text_skill import TextSkill

skills_directory = "../../semantic-kernel/samples/skills/"
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")
text_skill = kernel.import_skill(TextSkill(), "TextSkill")

Define your ASK. What do you want the Kernel to do?

# Basic Planner

 Let's start by taking a look at a basic planner. The `BasicPlanner` produces a JSON-based plan that aims to solve the provided ask sequentially and evaluated in order.

In [25]:
from semantic_kernel.planning.basic_planner import BasicPlanner

planner = BasicPlanner()

In [26]:
basic_plan = await planner.create_plan_async(ask, kernel)

In [27]:
print(basic_plan.generated_plan)

{
    "input": "Valentine's Day Date Ideas",
    "subtasks": [
        {"function": "WriterSkill.Brainstorm"},
        {"function": "WriterSkill.EmailGen", "args": {"input": "date_ideas"}},
        {"function": "WriterSkill.Translate", "args": {"language": "French"}},
        {"function": "TextSkill.uppercase"}
    ]
}


You can see that the Planner took my ask and converted it into an JSON-based plan detailing how the AI would go about solving this task, making use of the skills that the Kernel has available to it.

As you can see in the above plan, the AI has determined which functions to call in order to fulfill the user ask. The output of each step of the plan becomes the input to the next function.

Let's also define an inline skill and have it be available to the Planner. Be sure to give it a function name and skill name.

In [8]:
sk_prompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
"""
shakespeareFunction = kernel.create_semantic_function(
    prompt_template=sk_prompt,
    function_name="shakespeare",
    skill_name="ShakespeareSkill",
    max_tokens=2000,
    temperature=0.8,
)

Let's update our ask using this new skill

In [9]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. She speaks French so write it in French.
Convert the text to uppercase."""

new_plan = await planner.create_plan_async(ask, kernel)

In [10]:
print(new_plan.generated_plan)

{
    "input": "Valentine's Day Date Ideas",
    "subtasks": [
        {"function": "WriterSkill.Brainstorm"},
        {"function": "ShakespeareSkill.shakespeare"},
        {"function": "WriterSkill.Translate", "args": {"language": "French"}},
        {"function": "TextSkill.uppercase"}
    ]
}


### Executing the plan

Now that we have a plan, let's try to execute it! The Planner has a function called `execute_plan`.

In [11]:
results = await planner.execute_plan_async(new_plan, kernel)

In [12]:
print(results)

1. ÉCOUTEZ ! UN DÎNER AUX CHANDELLES, ENVELOPPÉ DANS DES ATOURS,
   DANS UN NOBLE RESTAURANT, OÙ LA FLAMME DE L'AMOUR VACILLERA DOUCEMENT.
2. UN PIQUE-NIQUE DANS LE PARC VERDOYANT, AVEC DU CHAMPAGNE ET DES FRAISES,
   OÙ NOUS NOUS ALLONGERONS SUR LE SOL DE LA NATURE ET NOUS DÉLECTERONS DE DOUCES MÉLODIES.
3. ASSISTER À UN COURS DE CUISINE, OÙ ENSEMBLE NOUS APPRENDRONS,
   UNE NOUVELLE RECETTE POUR AIGUISER NOS APPÉTITS ET ÉVEILLER LES PASSIONS.
4. PARTICIPER À UNE DÉGUSTATION DE VIN, DANS LE DOMAINE D'UN VIGNOBLE LOCAL,
   SAVOURER LE NECTAR DES DIEUX ET SENTIR L'ÉLIXIR DE L'AMOUR DANS CHAQUE VEINE.
5. UNE SOIRÉE CINÉMA À LA MAISON, AVEC UN FESTIN FAIT MAISON À PARTAGER,
   SOUS DES COUVERTURES DOUILLETTES, LAISSONS-NOUS BAIGNER DANS LE TENDRE ÉCLAT DE L'AMOUR.
6. UNE ESCAPADE SURPRISE LE WEEK-END, VERS UN ROYAUME ROMANTIQUE LOINTAIN,
   OÙ NOS CŒURS S'ÉLÈVERONT SUR LES AILES DE LA PASSION, COMME UNE ÉTOILE FILANTE.
7. SE LANCER DANS UNE RANDONNÉE PITTORESQUE, GRAVISSANT LES SOMMETS DU

# Workaround


In [20]:
new_plan.

semantic_kernel.planning.basic_planner.Plan

# The Plan Object Model

To build more advanced planners, we need to introduce a proper Plan object that can contain all the necessary state and information needed for high quality plans.

To see what that object model is, look at (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/plan.py)

# Sequential Planner

The sequential planner is an XML-based step-by-step planner. You can see the prompt used for it here (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/sequential_planner/Skills/SequentialPlanning/skprompt.txt)

In [14]:
from semantic_kernel.planning import SequentialPlanner

planner = SequentialPlanner(kernel)

In [15]:
sequential_plan = await planner.create_plan_async(goal=ask)

To see the steps that the Sequential Planner will take, we can iterate over them and print their descriptions

In [16]:
for step in sequential_plan._steps:
    print(step.description, ":", step._state.__dict__)

Given a goal or topic description generate a list of ideas : {'variables': {'input': ''}}
Generic function, unknown purpose : {'variables': {'input': ''}}
Translate the input into a language of your choice : {'variables': {'input': ''}}
Convert a string to uppercase. : {'variables': {'input': ''}}


Let's ask the sequential planner to execute the plan.

In [17]:
result = await sequential_plan.invoke_async()

In [18]:
print(result)

1. UN RASSEMBLEMENT JOYEUX SOUS LA CANOPÉE VERDOYANTE,
   OÙ L'ÉTREINTE DE LA NATURE ET LE RIRE SE COMBINENT.
2. DANS UNE UNION TENDRE, NOUS CONJURERONS UN FESTIN,
   NOS MAINS ENTRELACÉES, L'ART CULINAIRE DÉCHAÎNÉ.
3. À TRAVERS CETTE FORÊT LOINTAINE, NOUS MARCHERONS,
   À CÔTÉ DES RUISSEAUX BABILLANTS, OÙ RÈGNE LA SÉRÉNITÉ.
4. VERS UNE DEMEURE DES BEAUX-ARTS, NOUS NOUS AVENTURERONS,
   TÉMOIGNANT DE CHEFS-D'ŒUVRE QUI ÉMEUVENT L'ÂME.
5. UNE NUIT DE GAIETÉ ET DE PLAISANTERIE, SUR UNE SCÈNE COMIQUE,
   OÙ LE RIRE RÉSONNERA, ALORS QUE NOUS TOURNONS LA PAGE DE LA VIE.
6. UNE ESCAPADE PITTORESQUE, NOUS ERRERONS SUR LES ROUTES DE CAMPAGNE,
   DÉCOUVRANT DES TRÉSORS CACHÉS, NOS CŒURS SE SENTIRONT CHEZ EUX.
7. LAISSONS-NOUS EMBARQUER DANS UNE AVENTURE CULINAIRE INÉDITE,
   GOÛTANT DES SAVEURS EXOTIQUES, DES SECRETS ENCORE À DÉVOILER.
8. UNE DÉGUSTATION, TELLE LE NECTAR DES DIEUX, NOUS SAVOURERONS,
   APPRÉCIANT CHAQUE GORGÉE, ALORS QU'ELLE NOUS ÉVEILLE.
9. DANS UNE CHAMBRE D'OMBRES, NOS ÉCRANS

# Action Planner

The action planner takes in a list of functions and the goal, and outputs a **single** function to use that is appropriate to meet that goal.

In [19]:
from semantic_kernel.planning import ActionPlanner

planner = ActionPlanner(kernel)

Let's add more skills to the kernel

In [20]:
from semantic_kernel.core_skills import FileIOSkill, MathSkill, TextSkill, TimeSkill

kernel.import_skill(MathSkill(), "math")
kernel.import_skill(FileIOSkill(), "fileIO")
kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(TextSkill(), "text")

{'lowercase': SKFunction(),
 'trim': SKFunction(),
 'trim_end': SKFunction(),
 'trim_start': SKFunction(),
 'uppercase': SKFunction()}

In [21]:
ask = "What is the sum of 110 and 990?"

In [22]:
plan = await planner.create_plan_async(goal=ask)

In [23]:
result = await plan.invoke_async()

In [24]:
print(result)

1100


# Stepwise Planner

Stepwise Planner is based off the paper from MRKL (Modular Reasoning, Knowledge and Language) and is similar to other papers like ReACT (Reasoning and Acting in Language Models). At the core, the stepwise planner allows for the AI to form "thoughts" and "observations" and execute actions based off those to achieve a user's goal. This continues until all required functions are complete and a final output is generated.

See a video walkthrough of Stepwise Planner [here.](https://youtu.be/DG_Ge1v0c4Q?si=T1CHaAm1vV0mWRHu)

In [25]:
from semantic_kernel.planning import StepwisePlanner
from semantic_kernel.planning.stepwise_planner.stepwise_planner_config import (
    StepwisePlannerConfig,
)

Let's create a Bing Search native skill that we can pass in to the Kernel.

Make sure you have a Bing Search API key in your `.env` file

(https://www.microsoft.com/en-us/bing/apis/bing-web-search-api)

In [26]:
class WebSearchEngineSkill:
    """
    A search engine skill.
    """

    from semantic_kernel.orchestration.sk_context import SKContext
    from semantic_kernel.skill_definition import (
        sk_function,
        sk_function_context_parameter,
    )

    def __init__(self, connector) -> None:
        self._connector = connector

    @sk_function(description="Performs a web search for a given query", name="searchAsync")
    @sk_function_context_parameter(
        name="query",
        description="The search query",
    )
    async def search_async(self, query: str, context: SKContext) -> str:
        query = query or context.variables.get("query")[1]
        result = await self._connector.search_async(query, num_results=5, offset=0)
        return str(result)

In [27]:
from semantic_kernel.connectors.search_engine import BingConnector

BING_API_KEY = "debc8bdb85944e54b3199eec4213bf55"
connector = BingConnector(BING_API_KEY)
kernel.import_skill(WebSearchEngineSkill(connector), skill_name="WebSearch")

{'searchAsync': SKFunction()}

Let's also add a couple more skills

In [28]:
from semantic_kernel.core_skills.math_skill import MathSkill
from semantic_kernel.core_skills.time_skill import TimeSkill

kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(MathSkill(), "math")

{'Add': SKFunction(), 'Subtract': SKFunction()}

In [29]:
planner = StepwisePlanner(kernel, StepwisePlannerConfig(max_iterations=10, min_iteration_time_ms=1000))

Now let's do a more complicated ask that will require planner to make a call to Bing to get the latest information.

In [30]:
ask = """What is the Argentinian inflation rate right now?"""

plan = planner.create_plan(goal=ask)

In [31]:
for step in plan._steps:
    print(step.description, ":", step._state.__dict__)

Execute a plan : {'variables': {'input': ''}}


In [32]:
result = await plan.invoke_async()

Variable `$agent_scratch_pad` not found


In [33]:
print(result)

The current Argentinian inflation rate is [insert the observed value here].


Let's see the steps that the AI took to get to the answer.

In [34]:
for index, step in enumerate(plan._steps):
    print("Step:", index)
    print("Description:", step.description)
    print("Function:", step.skill_name + "." + step._function.name)
    if len(step._outputs) > 0:
        print("  Output:\n", str.replace(result[step._outputs[0]], "\n", "\n  "))

Step: 0
Description: Execute a plan
Function: StepwisePlanner.ExecutePlan
  Output:
 This was my previous work (but they haven't seen any of it! They only see what I return as final answer):
  [THOUGHT]
  None


# Conclusion

- Planner and Sequential-Planner works in the documentation example
- Sequential-Planner fail many times, in one experimt due to OpenAI-gpt token quota, and other strange answer.